In [1]:
import pandas as pd
import numpy as np
import re
import json


from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [2]:
DATA_IN_PATH='./data_in2/'
train_data=pd.read_csv(DATA_IN_PATH+'train.csv',encoding='utf-8')

In [3]:
train_pos_data = train_data.loc[train_data['is_duplicate'] == 1]
train_neg_data = train_data.loc[train_data['is_duplicate'] == 0]

class_difference = len(train_neg_data) - len(train_pos_data)
sample_frac = 1 - (class_difference / len(train_neg_data))

train_neg_data = train_neg_data.sample(frac = sample_frac)

In [4]:
print("중복 질문 개수: {}".format(len(train_pos_data)))
print("중복이 아닌 질문 개수: {}".format(len(train_neg_data)))

중복 질문 개수: 149263
중복이 아닌 질문 개수: 149263


In [5]:
train_data=pd.concat([train_neg_data,train_pos_data])

In [6]:
FILTERS="([~.,!?\"':;'])"

change_filter=re.compile(FILTERS)

questions1=[str(s) for s in train_data['question1']]
questions2=[str(s) for s in train_data['question2']]

filtered_questions1=list()
filtered_questions2=list()

for q in questions1:
    filtered_questions1.append(re.sub(change_filter, "",q).lower())

for q in questions2:
    filtered_questions2.append(re.sub(change_filter, "",q).lower())


In [9]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(filtered_questions1+filtered_questions2)

In [10]:
questions1_sequence=tokenizer.texts_to_sequences(filtered_questions1)
questions2_sequence=tokenizer.texts_to_sequences(filtered_questions2)

In [14]:
MAX_SEQUENCE_LENGTH=31

q1_data=pad_sequences(questions1_sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
q2_data=pad_sequences(questions2_sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [17]:
word_vocab={}
word_vocab=tokenizer.word_index

labels=np.array(train_data['is_duplicate'], dtype=int)

print('Shape of question1 data: {}'.format(q1_data.shape))
print('Shape of question2 data: {}'.format(q2_data.shape))
print('Shape of label: {}'.format(labels.shape))
print('Words in index: {}'.format(len(word_vocab)))

Shape of question1 data: (298526, 31)
Shape of question2 data: (298526, 31)
Shape of label: (298526,)
Words in index: 75938


In [18]:
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size']=len(word_vocab)+1

In [19]:
TRAIN_Q1_DATA = 'train_q1.npy'
TRAIN_Q2_DATA = 'train_q2.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

np.save(open(DATA_IN_PATH + TRAIN_Q1_DATA, 'wb'), q1_data)
np.save(open(DATA_IN_PATH + TRAIN_Q2_DATA , 'wb'), q2_data)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA , 'wb'), labels)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))

In [20]:
test_data=pd.read_csv(DATA_IN_PATH+'test.csv', encoding='utf-8')
valid_ids=[type(x)==int for x in test_data.test_id]
test_data=test_data[valid_ids].drop_duplicates()

C:\Users\yuhwa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
test_questions1=[str(s) for s in test_data['question1']]
test_questions2=[str(s) for s in test_data['question2']]

filtered_test_questions1=list()
filtered_test_questions2=list()

for q in test_questions1:
    filtered_test_questions1.append(re.sub(change_filter,"",q).lower())

for q in test_questions2:
    filtered_test_questions2.append(re.sub(change_filter,"",q).lower())

In [26]:
test_questions1_sequence=tokenizer.texts_to_sequences(filtered_test_questions1)
test_questions2_sequence=tokenizer.texts_to_sequences(filtered_test_questions2)

test_q1_data=pad_sequences(test_questions1_sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
test_q2_data=pad_sequences(test_questions2_sequence, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [27]:
test_id=np.array(test_data['test_id'])

print('Shape of question1 data: {}'.format(test_q1_data.shape))
print('Shape of question2 data: {}'.format(test_q2_data.shape))
print('Shape of ids: {}'.format(test_id.shape))

Shape of question1 data: (2345796, 31)
Shape of question2 data: (2345796, 31)
Shape of ids: (2345796,)


In [28]:
TEST_Q1_DATA = 'test_q1.npy'
TEST_Q2_DATA = 'test_q2.npy'
TEST_ID_DATA = 'test_id.npy'

np.save(open(DATA_IN_PATH + TEST_Q1_DATA, 'wb'), test_q1_data)
np.save(open(DATA_IN_PATH + TEST_Q2_DATA , 'wb'), test_q2_data)
np.save(open(DATA_IN_PATH + TEST_ID_DATA , 'wb'), test_id)